In [1]:

import os
import sys
import sklearn
import pip
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import deeplake
from time import time
from PIL import Image


from pathlib import Path
# set figure size
plt.rcParams['figure.figsize'] = [14, 14]
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Requirements taken from https://www.tensorflow.org/install/pip#linux
python_version = sys.version
pip_version = pip.__version__
nvidia_smi_version = os.popen('nvidia-smi --query-gpu=driver_version --format=csv,noheader').read().strip()
cuda_version = os.popen('nvcc --version').read().split('\n')[3].split(',')[1].strip()
tensorflow_version = tf.__version__
physical_devices = tf.config.list_physical_devices('GPU')
num_gpus = len(physical_devices)
gpu_model = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()

req_xray_data = pd.DataFrame({
    'Package': ['Python', 'pip', 'nvidia-smi', 'cuda', 'tensorflow', 'GPUs'],
    'Required': ['3.7', '20.2', '450.51.06', '11.0', '2.3.0', '1'],
    'Installed': [python_version, pip_version, nvidia_smi_version, cuda_version, tensorflow_version, num_gpus]
})
               
print(f'All requiremetns met. Using TF with GPU: {gpu_model} ({num_gpus}x)')

pd.options.plotting.backend = "plotly"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

from keras.models import Model, Sequential
from keras.layers import Lambda, Input, GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import keras
from sklearn.metrics import classification_report, accuracy_score

import sys

sys.path.append('/home/kayaba_attribution/Documents/UoL/FINAL_PROJECT/Code/nih-chest')
sys.path.append(os.path.dirname(os.getcwd()))

from dataset import train_ds, val_ds, test_ds, y_test, y_val, ALL_LABELS, IMG_SIZE
from reportUtils import generate_report

MODEL_NAME = 'VGGNet16'

req_xray_data


2024-03-10 22:04:58.526844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 22:04:58.526880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 22:04:58.527795: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 22:04:58.534496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 22:04:59.169893: W tensorflow/compiler/tf2

All requiremetns met. Using TF with GPU: NVIDIA GeForce GTX 1660 (1x)
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660, compute capability 7.5


2024-03-10 22:05:00.395686: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355



Train size: X: 61540, y: 61540 | Validation size: X: 15385, y: 15385 | Test size: X: 35195, y: 35195 | 
Total size: X: 112120


2024-03-10 22:05:00.700966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-10 22:05:00.701193: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-10 22:05:00.701347: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

,Package,Required,Installed
0,Python,3.7,"3.9.17 (main, Jul 5 2023, 20:41:20) \n[GCC 11..."
1,pip,20.2,23.2.1
2,nvidia-smi,450.51.06,535.54.03
3,cuda,11.0,release 12.2
4,tensorflow,2.3.0,2.15.0
5,GPUs,1,1


In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model

#### VGGNet-16


In [3]:
input_layer = Input((IMG_SIZE,IMG_SIZE,3)) 

conv1  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(input_layer)
conv2  = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

gap    = GlobalAveragePooling2D()(pool4)

dense1 = Dense(2048, activation="relu")(gap)
dense2 = Dense(2048, activation="relu")(dense1)
output = Dense(15, activation="softmax")(dense2)

model = Model(inputs=input_layer, outputs=output)


plot_model(model, f"{MODEL_NAME}_model.png", show_shapes=True, show_layer_names=True)
weight_path=f"{MODEL_NAME}_weights.best.hdf5".format('Basic_CNN')

# https://keras.io/api/callbacks/model_checkpoint/
checkpoint = ModelCheckpoint(
    weight_path,
    monitor='val_loss',
    verbose=1, 
    save_best_only=True,
    mode='min',
    save_weights_only = True)

# https://keras.io/api/callbacks/early_stopping/
earlystop = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 1e-4,
    patience = 5,
    mode = 'min', 
    restore_best_weights = True,
    verbose = 1)

# https://keras.io/api/callbacks/reduce_lr_on_plateau/
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    verbose=1,
    mode='auto',
    min_delta=1e-4,
    cooldown=1,
    min_lr=1e-6)


callbacks_list = [checkpoint, earlystop, reduceLROnPlat]


model.compile(optimizer='adam', 
              loss=keras.losses.BinaryCrossentropy(label_smoothing=0.0), 
                metrics=[
                    keras.metrics.BinaryAccuracy(name='accuracy'),
                    keras.metrics.Precision(name='precision'),
                    keras.metrics.Recall(name='recall'),
                    keras.metrics.AUC(name='auc', multi_label=True)])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      36928     
                                                             

In [4]:
# Check if weight checkpoint exists
if os.path.exists(weight_path):
    print("Loading weights from:", weight_path)
    model.load_weights(weight_path)
else:
    print("No weights found, starting training from scratch.")

start = time()
history = model.fit(
                    train_ds,
                    epochs=5,
                    validation_data=val_ds,
                    callbacks=callbacks_list
                    )
print('\nTraining took {} sec'.format((time()-start)))
model.save(f"{MODEL_NAME}.keras")


No weights found, starting training from scratch.
Epoch 1/5


2024-03-10 22:05:03.698712: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-10 22:06:37.032965: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7f300042f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-10 22:06:37.033019: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2024-03-10 22:06:37.040165: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710122797.146504 3680968 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 102/1924 [>.............................] - ETA: 8:12 - loss: 0.3381 - accuracy: 0.8960 - precision: 0.2400 - recall: 0.0024 - auc: 0.5030

2024-03-10 22:07:06.627130: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


 716/1924 [==========>...................] - ETA: 5:24 - loss: 0.3042 - accuracy: 0.8963 - precision: 0.2400 - recall: 3.3701e-04 - auc: 0.5002

2024-03-10 22:09:51.120674: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


 959/1924 [=============>................] - ETA: 4:19 - loss: 0.3031 - accuracy: 0.8963 - precision: 0.2400 - recall: 2.5152e-04 - auc: 0.5010

2024-03-10 22:10:56.837091: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


1924/1924 [==============================] - ETA: 0s - loss: 0.3007 - accuracy: 0.8961 - precision: 0.4429 - recall: 6.4639e-04 - auc: 0.5139
Epoch 1: val_loss improved from inf to 0.29382, saving model to VGGNet16_weights.best.hdf5
1924/1924 [==============================] - 679s 302ms/step - loss: 0.3007 - accuracy: 0.8961 - precision: 0.4429 - recall: 6.4639e-04 - auc: 0.5139 - val_loss: 0.2938 - val_accuracy: 0.8970 - val_precision: 0.5158 - val_recall: 0.0069 - val_auc: 0.5686 - lr: 0.0010
Epoch 2/5
  48/1924 [..............................] - ETA: 8:14 - loss: 0.2948 - accuracy: 0.8968 - precision: 0.6364 - recall: 0.0029 - auc: 0.5316

KeyboardInterrupt: 

In [5]:
generate_report(model, val_ds, y_val, MODEL_NAME, ALL_LABELS)

481/481 [==============================] - 34s 71ms/step


ValueError: Output dtype not compatible with inputs.